# Data Preprocessing #

Now that we know precisely which weather variables are included in the GRIB2 data, we can start processing our xarray dataset.

As in the last section, **Data Inspecting**, we get these weather data: 
- `latitude (degree(-90 to +90))`
- `longitude (degree(0 to 360))`
- `temperature at ground or water surface (K)`
- `U-component at isobaric surface (Pa) (m/s) `
- `V-component at isobaric surface (Pa) (m/s) ` 

From these lookup key `TMP_P0_L1_GLL0`, `UGRD_P0_L100_GLL0`, `VGRD_P0_L100_GLL0`

But, the data format that we want are:
- `latitude (degree(-90 to +90))`
- `longitude (degree(-180 to +180))`
- `surface temperature(°c)`
- `wind speed at 85000.0 Pa (km/hr)` 
- `wind direction at 85000.0 Pa (degree(0-360))`
- `datetime` as an index of the dataset
- **We want only the data in Thailand**

As you can see, almost all of them are not in the right format. So, in this section, we will preprocess the data to be in the desirable format!

# Import Libraries

In [1]:
import Nio
import xarray as xr
import math
import glob
import pandas as pd
from datetime import datetime, timedelta

# Open Dataset

In [2]:
path = "data/gfsanl_4_20171107_0000_000.grb2"

ds = xr.open_dataset(path, engine="pynio")
df = ds.get(["TMP_P0_L1_GLL0", "UGRD_P0_L100_GLL0", "VGRD_P0_L100_GLL0"]).to_dataframe()

# Latitude and Longitude

Although latitude values are already in the standard range of -90 degress to +90 degrees, longitude values are in the range of 0 to +360.

To make the data easier to work with, we convert longitude values into the standard range of -180 degrees to +180 degrees

In [3]:
latitudes = df.index.get_level_values("lat_0")
longitudes = df.index.get_level_values("lon_0")

map_function = lambda lon: (lon - 360) if (lon > 180) else lon
remapped_longitudes = longitudes.map(map_function)

df["longitude"] = remapped_longitudes
df["latitude"] = latitudes  

In [4]:
df

TMP_P0_L1_GLL0  UGRD_P0_L100_GLL0  VGRD_P0_L100_GLL0  \
lat_0 lon_0 lv_ISBL0                                                         
 90.0 0.0   100.0         247.178085         -38.843361           4.000019   
            200.0         247.178085         -35.828548           1.100024   
            300.0         247.178085         -36.096478           0.199982   
            500.0         247.178085         -34.036407          -0.800006   
            700.0         247.178085         -31.010010          -0.700012   
...                              ...                ...                ...   
-90.0 359.5 90000.0       243.478073           1.780039          -6.711587   
            92500.0       243.478073           1.779717          -6.711902   
            95000.0       243.478073           1.776223          -6.710415   
            97500.0       243.478073           1.772436          -6.710183   
            100000.0      243.478073           1.778059          -6.710752   

                      longitude  latitude  
lat_0 lon_0 lv_ISBL0                       
 90.0 0.0   100.0           0.0      90.0  
            200.0           0.0      90.0  
            300.0           0.0      90.0  
            500.0           0.0      90.0  
            700.0           0.0      90.0  
...                         ...       ...  
-90.0 359.5 90000.0        -0.5     -90.0  
            92500.0        -0.5     -90.0  
            95000.0        -0.5     -90.0  
            97500.0        -0.5     -90.0  
            100000.0       -0.5     -90.0  

[8057520 rows x 5 columns]

As one of our requirements is that "**We want only the data in Thailand**"

The next step in the process is defining our geospatial bounding box, which we do by specifying the minimum and maximum 
coordinate values

You can get the country bounding boxes from [here](https://wiki.openstreetmap.org/wiki/User:Ewmjc/Country_bounds) or [here](https://gist.github.com/graydon/11198540)

In [5]:
min_lat = 5.616667
max_lat = 20.442778
min_lon = 97.366667
max_lon = 105.766667

In [6]:
lat_filter = (df["latitude"] >= min_lat) & (df["latitude"] <= max_lat)
lon_filter = (df["longitude"] >= min_lon) & (df["longitude"] <= max_lon)

# Surface Temperature

As we need only the temperature that at the specific pressure which is `85000.0 Pa`, the next step is to choose only the desirable temperature

In [7]:
specific_press = 85000.0

In [8]:
try:
    press_filter = df.index.get_level_values("lv_ISBL0") == specific_press
except:
    press_filter = df.index.get_level_values("lv_ISBL5") == specific_press

And also convert the temperature unit from `K` to `°c`

So at this point, the dataframe would look like this

In [9]:
df = df.loc[lat_filter & lon_filter & press_filter]   
df = df.reset_index(drop=True)
df.columns = ["temperature", "u_component", "v_component", "longitude", "latitude"]
df['temperature'] = df['temperature'].apply(lambda k: k - 273.15)

In [10]:
df

,temperature,u_component,v_component,longitude,latitude
0,13.128076,-2.611416,-2.778105,97.5,20.0
1,15.928094,-2.601416,-1.628105,98.0,20.0
2,16.128076,-2.781416,-1.028105,98.5,20.0
3,15.828088,-2.541416,-1.208105,99.0,20.0
4,16.028070,-5.131416,-4.308105,99.5,20.0
...,...,...,...,...,...
488,28.228082,-1.121416,2.731894,103.5,6.0
489,28.228082,-2.321416,3.011894,104.0,6.0
490,28.128076,-2.971416,3.441895,104.5,6.0
491,28.028070,-2.551416,3.281894,105.0,6.0


# Wind Speed and Wind Direction

We can find `wind speed` and `wind direction` from `U-component` and `V-component`

The unit of each variable are shown below:
- `wind speed` : km/hr
- `wind direction` : degree(0-360)
- `U-component` : m/s
- `V-component` : m/s

In [11]:
def find_wind_speed_and_direction(u, v):
    if u == 0 and v == 0:
        return 0, 0
    
    wind_speed = math.sqrt(u**2 + v**2)
    wind_speed *= 18/5
    
    wind_direction = math.atan2(u/wind_speed, v/wind_speed) + math.pi
    wind_direction *= 180/math.pi
    
    return wind_speed, wind_direction

In [12]:
df[["wind_speed", "wind_direction"]] = df[["u_component", "v_component"]].apply(lambda x: find_wind_speed_and_direction(*x), axis=1, result_type="expand")

# Datetime

We can get `datetime` from the `filename` of the dataset

The example of `filename`: 
- `gfsanl_4_20200515_0000_000.grb2`
- `/mnt/c/Users/non_s/Desktop/gfsanl_4_20200515_0000_000.grb2`

As we want only the data in Thailand, we need to convert the datetime of filename from `UTC` to `UTC+7`

In [13]:
def get_datetime(path):
    tmp = path.split("/")[-1]
    tmp = tmp.split(".")[0]
    tmp = tmp.split("_")
    
    date = tmp[2]
    hour = tmp[3][:2]
    
    year = date[:4]
    month = date[4:6]
    day = date[6:]
    
    _datetime = datetime(int(year),int(month),int(day),int(hour)) + timedelta(hours=7)
    
    return _datetime

In [14]:
df["datetime"] = get_datetime(path)

Drop unwanted columns and rename column to the meaningful name, the final output would look like this

In [15]:
df = df.drop(['u_component', 'v_component'], axis=1)
df = df[["datetime", "latitude", "longitude", "temperature", "wind_speed", "wind_direction"]]

In [16]:
df

,datetime,latitude,longitude,temperature,wind_speed,wind_direction
0,2017-11-07 07:00:00,20.0,97.5,13.128076,13.726042,43.228494
1,2017-11-07 07:00:00,20.0,98.0,15.928094,11.048007,57.959462
2,2017-11-07 07:00:00,20.0,98.5,16.128076,10.675245,69.713981
3,2017-11-07 07:00:00,20.0,99.0,15.828088,10.130219,64.575124
4,2017-11-07 07:00:00,20.0,99.5,16.028070,24.120323,49.984695
...,...,...,...,...,...,...
488,2017-11-07 07:00:00,6.0,103.5,28.228082,10.631173,157.682329
489,2017-11-07 07:00:00,6.0,104.0,28.228082,13.689698,142.376782
490,2017-11-07 07:00:00,6.0,104.5,28.128076,16.369494,139.195685
491,2017-11-07 07:00:00,6.0,105.0,28.028070,14.965159,142.137731


# Complete Code

In [17]:
def find_wind_speed_and_direction(u, v):
    if u == 0 and v == 0:
        return 0, 0
    
    wind_speed = math.sqrt(u**2 + v**2)
    wind_speed *= 18/5
    
    wind_direction = math.atan2(u/wind_speed, v/wind_speed) + math.pi
    wind_direction *= 180/math.pi
    
    return wind_speed, wind_direction

In [18]:
def get_datetime(path):
    tmp = path.split("/")[-1]
    tmp = tmp.split(".")[0]
    tmp = tmp.split("_")
    
    date = tmp[2]
    hour = tmp[3][:2]
    
    year = date[:4]
    month = date[4:6]
    day = date[6:]
    
    _datetime = datetime(int(year),int(month),int(day),int(hour)) + timedelta(hours=7)
    
    return _datetime

In [19]:
def create_dt(path):
    ds = xr.open_dataset(path, engine="pynio")
    
    df = ds.get(["TMP_P0_L1_GLL0", "UGRD_P0_L100_GLL0", "VGRD_P0_L100_GLL0"]).to_dataframe()
    
    latitudes = df.index.get_level_values("lat_0")
    longitudes = df.index.get_level_values("lon_0")
    
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    
    df["longitude"] = remapped_longitudes
    df["latitude"] = latitudes    
    
    min_lat = 5.616667
    max_lat = 20.442778
    min_lon = 97.366667
    max_lon = 105.766667
    specific_press = 85000.0
    
    lat_filter = (df["latitude"] >= min_lat) & (df["latitude"] <= max_lat)
    lon_filter = (df["longitude"] >= min_lon) & (df["longitude"] <= max_lon)
    
    try:
        press_filter = df.index.get_level_values("lv_ISBL0") == specific_press
    except:
        press_filter = df.index.get_level_values("lv_ISBL5") == specific_press
    
    df = df.loc[lat_filter & lon_filter & press_filter]   
    df = df.reset_index(drop=True)
    df.columns = ["temperature", "u_component", "v_component", "longitude", "latitude"]
    
    df[["wind_speed", "wind_direction"]] = df[["u_component", "v_component"]].apply(lambda x: find_wind_speed_and_direction(*x), axis=1, result_type="expand")
    df['temperature'] = df['temperature'].apply(lambda k: k - 273.15)
    df["datetime"] = get_datetime(path)
    
    df = df.drop(['u_component', 'v_component'], axis=1)
    df = df[["datetime", "latitude", "longitude", "temperature", "wind_speed", "wind_direction"]]
    
    return df

In [20]:
path = "data/gfsanl_4_20171107_0000_000.grb2"
create_dt(path)

,datetime,latitude,longitude,temperature,wind_speed,wind_direction
0,2017-11-07 07:00:00,20.0,97.5,13.128076,13.726042,43.228494
1,2017-11-07 07:00:00,20.0,98.0,15.928094,11.048007,57.959462
2,2017-11-07 07:00:00,20.0,98.5,16.128076,10.675245,69.713981
3,2017-11-07 07:00:00,20.0,99.0,15.828088,10.130219,64.575124
4,2017-11-07 07:00:00,20.0,99.5,16.028070,24.120323,49.984695
...,...,...,...,...,...,...
488,2017-11-07 07:00:00,6.0,103.5,28.228082,10.631173,157.682329
489,2017-11-07 07:00:00,6.0,104.0,28.228082,13.689698,142.376782
490,2017-11-07 07:00:00,6.0,104.5,28.128076,16.369494,139.195685
491,2017-11-07 07:00:00,6.0,105.0,28.028070,14.965159,142.137731


# Processing Multiple Data Files

It is often desirable to process multiple data files at once, in order to combine the results into a single unified CSV output file.

In [21]:
def main(path = "data"):
    path_list = sorted(glob.glob(path +'/*.grb2'))
    main_helper(path_list)
    return None

In [22]:
def main_helper(path_list):
    output_df = pd.DataFrame()
    for path in path_list:
        try:
            print("-- Processing {} --".format(path))
            data = create_dt(path)
            if data is not None:
                output_df = pd.concat([output_df, data])
            else:
                print("{} is None".format(path))
        except Exception as e:
            print(e)
            continue
    print("Finished!")
    output_df.to_csv("output_data.csv", index=False)
    return None

In [23]:
main()

-- Processing data/gfsanl_4_20171107_0000_000.grb2 --
-- Processing data/gfsanl_4_20200515_0000_000.grb2 --
Finished!


In [24]:
output = pd.read_csv("output_data.csv")
output

,datetime,latitude,longitude,temperature,wind_speed,wind_direction
0,2017-11-07 07:00:00,20.0,97.5,13.128076,13.726042,43.228494
1,2017-11-07 07:00:00,20.0,98.0,15.928094,11.048007,57.959462
2,2017-11-07 07:00:00,20.0,98.5,16.128076,10.675245,69.713981
3,2017-11-07 07:00:00,20.0,99.0,15.828088,10.130219,64.575124
4,2017-11-07 07:00:00,20.0,99.5,16.028070,24.120323,49.984695
...,...,...,...,...,...,...
981,2020-05-15 07:00:00,6.0,103.5,29.850000,10.114761,116.447935
982,2020-05-15 07:00:00,6.0,104.0,29.749994,9.304468,131.168921
983,2020-05-15 07:00:00,6.0,104.5,29.650018,10.201203,129.210855
984,2020-05-15 07:00:00,6.0,105.0,29.749994,11.297353,114.500117
